In [1]:
using PyPlot, Printf, Random, DelimitedFiles, Dates

include("decode.jl")
include("utils.jl")
include("updateRule.jl");

In [2]:
Random.seed!(1234);

In [3]:
function computeSBFAndTheta(rule, x_t, col)
   return rule.computeSBF(x_t, col, alpha_0[:,col])
end

function decodeSurprise(seq, m, alpha_0, rule, ignoreFirstM = true)
    function computeSBFAndTheta(rule, x_t, col)
        sbf = rule.computeSBF(x_t, col, alpha_0[:,col])
        theta = 1 - rule.computeTheta(col)
        return (sbf, theta) 
    end
    
    # define callback to compute SBF and Thetas
    returnType = Tuple{Float64, Float64}
    callback = Callback(computeSBFAndTheta, returnType)

    # decode sequence
    values = decode(seq, m, alpha_0, rule;
                    callback = callback, ignoreFirstM = ignoreFirstM)
    
    if ignoreFirstM && m > 0
        values[1:m] .= Ref{returnType}((1.0, 0.5))
    end
    
    return values
end;

In [4]:
function generateSequenceBase(chunks, ps)
    seq = Array{Int32}(undef, sum(chunks))
    prob = Array{Float32}(undef, sum(chunks))
    
    i = 1
    for (p, chunk) in zip(ps, chunks)
        for j = 1:chunk
            seq[i] = rand() < p
            prob[i] = p
            i += 1
        end
    end
    
    return (seq, prob)
end;

In [5]:
function generateSequenceTrans(chunks, ps1g2, ps2g1)
    seq = Array{Int32}(undef, sum(chunks))
    prob1g2 = Array{Float32}(undef, sum(chunks))
    prob2g1 = Array{Float32}(undef, sum(chunks))
    
    i = 2
    
    # set inital value
    seq[1] = rand() > 0.5
    prob1g2[1] = ps1g2[1]
    prob2g1[1] = ps2g1[1]
    
    for (p1g2, p2g1, chunk) in zip(ps1g2, ps2g1, chunks)
        for j = 1:chunk
            p = seq[i - 1] == 0 ? p2g1 : 1 - p1g2
            seq[i] = rand() < p
            prob1g2[i] = p1g2
            prob2g1[i] = p2g1
            i += 1
            if i > sum(chunks)
               break 
            end
        end
    end
    
    return (seq, prob1g2, prob2g1)
end;

In [6]:
function exportSeq(seq, filename)
    f = open(filename, "w")

    for el in seq
        print(f, el)
        print(f, " ")
    end
    
    close(f)
end
;

In [7]:
function readThetasMatlab(filename, len)
   f = open(filename, "r")
    t = readdlm(f)
    close(f)
    return t[1,1:len]
end;

In [8]:
function readSeq(filename, len)
    f = open(filename, "r")
    t = readdlm(f)
    close(f)
    
    return t[1,1:len]
end;

In [9]:
# ============== DECODE BASE RATE ======================
# generate sequence
chunks = [200, 200, 200, 200, 200]
len = sum(chunks)

p = 0.25
ps = [p, 1-p, p, 1-p, p]

seq_base = readSeq("../test/seq1.txt", 1000)
# (seq_base, prob) = generateSequenceBase(chunks, ps)
# exportSeq(seq_base, "../test/seq1.txt")
# exportSeq(prob, "../test/seq1_p.txt")

"""
# matlab code

s = dlmread('seq1.txt') + 1; % add one because they work with elements 1 and 2
gen_p1 = dlmread('seq1_p.txt');

% compute ...

fid = fopen('seq1_leaky_10_theta.txt','wt'); % change filename
fprintf(fid, '%.9f ', out.FIX.p1_mean');
fclose(fid);

return;
"""
;

In [10]:
values = decodeSurprise(seq_base, 0, ones(2, 1), perfect());

# here we do 2:end
thetas = map(t -> t[2], values[2:end]);
thetas_matlab = readThetasMatlab("../test/seq1_perfect_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [14]:
values = decodeSurprise(seq_base, 0, ones(2, 1), leaky(10));

thetas = map(t -> t[2], values[2:end]);
thetas_matlab = readThetasMatlab("../test/seq1_leaky_10_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [15]:
# ============== DECODE TRANSITION RATE ======================
# generate sequence
chunks = [100, 200, 100, 200]
len = sum(chunks)

p = 0.25
p1g2  = 1 .- [p, 1-p, p, 1-p]
p2g1  =      [p, 1-p, p, 1-p]

seq_trans = readSeq("../test/seq2.txt", 600);
# (seq_trans, prob1g2, prob2g1) = generateSequenceTrans(chunks, p1g2, p2g1)
# exportSeq(seq, "seq2.txt")
# exportSeq(prob1g2, "seq2_p1g2.txt")
# exportSeq(prob2g1, "seq2_p2g1.txt")

In [23]:
rule = perfect()
values = decodeSurprise(seq_trans, 1, ones(2, 2), rule)

# here we do 2:end because we want to skip the first value (incomplete window)
thetas = map(t -> t[2], values[2:end]);
thetas_matlab = readThetasMatlab("../test/seq2_perfect_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [17]:
rule = leaky(16, true)
values = decodeSurprise(seq_trans, 1, ones(2, 2), rule);

thetas = map(t -> t[2], values[2:end]);
thetas_matlab = readThetasMatlab("../test/seq2_leaky_16_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [20]:
seq_trans

600-element Array{Any,1}:
 1
 1
 0
 1
 0
 0
 0
 1
 0
 1
 1
 0
 1
 ⋮
 1
 1
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1

In [26]:
1 - rule.computeTheta(2)

0.31518624641833815

In [22]:
thetas_matlab

600-element Array{Any,1}:
 0.5        
 0.333333333
 0.5        
 0.5        
 0.333333333
 0.5        
 0.6        
 0.6        
 0.5        
 0.666666667
 0.571428571
 0.428571429
 0.625      
 ⋮          
 0.321533923
 0.320588235
 0.319648094
 0.31871345 
 0.56916996 
 0.320699708
 0.319767442
 0.31884058 
 0.317919075
 0.317002882
 0.316091954
 0.315186246

In [19]:
thetas

600-element Array{Float64,1}:
 0.5                
 0.340203972232077  
 0.5                
 0.5072029381048466 
 0.3469214482427885 
 0.5146458323269223 
 0.6133639169419289 
 0.5997682024060336 
 0.5036738645033848 
 0.6692478170999395 
 0.5516191379688065 
 0.42701478744354116
 0.6156732547626202 
 ⋮                  
 0.23743159017919524
 0.2234943524498194 
 0.21084580091974048
 0.19933659155735317
 0.4348710727133406 
 0.2564090657626745 
 0.24115299497766263
 0.2272862693559855 
 0.21465097754603246
 0.20311164397781967
 0.19255137866072647
 0.18286879489831187